In [4]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import multiprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import json
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModel

In [5]:
data_source = pd.read_csv('data_source.csv')
#data_source = data_source[data_source['label']=='153 - Input Data Manipulation']
data_source, _ = train_test_split(data_source, test_size=0.75, random_state=42)
data_target  = pd.read_csv('data_target.csv')
#data_target, _ = train_test_split(data_target, test_size=0.9, random_state=42)

data_target = data_target.sample(frac=1, random_state=42)
#data_target = data_target[data_target['label']=='153 - Input Data Manipulation']
X_source = data_source['text'].str.replace('/',' ')
y_source = data_source['label']
X_target  = data_target ['text'].str.replace('/','')
y_target  = data_target ['label']
rlist =['194 - Fake the Source of Data', '66 - SQL Injection',
       '34 - HTTP Response Splitting', '126 - Path Traversal',
       '000 - Normal', '272 - Protocol Manipulation',
       '310 - Scanning for Vulnerable Software', '242 - Code Injection',
       '153 - Input Data Manipulation']
mapping = {l: i+1 for i, l in enumerate(rlist)}
y_source = [mapping[s] for s in y_source] 
y_target  = [mapping[r] for r in y_target ]
y_source = np.array(y_source)
y_target  = np.array(y_target )


In [6]:
print(len(X_source))
print(len(X_target))

33145
31356


In [7]:
model_name = 'jackaduma/SecBERT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

X_source = list(X_source)
X_target = list(X_target)
input_texts_source = X_source
input_text_target = X_target

X_train = []
X_test = []

for i , text in enumerate(input_texts_source):

    tokens = tokenizer.tokenize(text)

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        outputs = model(input_tensor)

    last_hidden_state = outputs.last_hidden_state

    first_token_features = last_hidden_state[0, 0, :]

    X_train.append(first_token_features)
    if i%1000==0 :
        print(i/len(X_source))

for j , text in enumerate(input_text_target):

    tokens = tokenizer.tokenize(text)

    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        outputs = model(input_tensor)

    last_hidden_state = outputs.last_hidden_state

    first_token_features = last_hidden_state[0, 0, :]

    X_test.append(first_token_features)
    if j%1000==0 :
        print(j/len(X_target))


Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.0
0.03017046311660884
0.06034092623321768
0.09051138934982653
0.12068185246643535
0.1508523155830442
0.18102277869965305
0.21119324181626187
0.2413637049328707
0.27153416804947955
0.3017046311660884
0.33187509428269724
0.3620455573993061
0.3922160205159149
0.42238648363252373
0.4525569467491326
0.4827274098657414
0.5128978729823502
0.5430683360989591
0.573238799215568
0.6034092623321768
0.6335797254487856
0.6637501885653945
0.6939206516820033
0.7240911147986122
0.754261577915221
0.7844320410318298
0.8146025041484387
0.8447729672650475
0.8749434303816563
0.9051138934982652
0.9352843566148741
0.9654548197314828
0.9956252828480917
0.0
0.031891822936599055
0.06378364587319811
0.09567546880979717
0.12756729174639622
0.15945911468299528
0.19135093761959435
0.22324276055619338
0.25513458349279244
0.2870264064293915
0.31891822936599057
0.3508100523025896
0.3827018752391887
0.41459369817578773
0.44648552111238676
0.47837734404898585
0.5102691669855849
0.542160989922184
0.574052812858783
0.605

In [8]:
X_test = torch.stack(X_test)
X_train = torch.stack(X_train)

In [9]:
y_target = np.where(np.logical_or(y_target == 8, y_target == 9), 0, 1)

In [10]:
# class Vectorizer:
#     def __init__(self, method='BOW', ngram_range=(1, 1), max_features=300, emb_fname='', word_index_fname=''):
#         self.method = method
#         if self.method == 'BOW':
#             self.vectorizer = CountVectorizer(analyzer='word', input='content', ngram_range=ngram_range, max_features=max_features)
#         elif self.method == 'TFIDF':
#             self.vectorizer = TfidfVectorizer(analyzer='word', input='content', max_features=max_features)
#         elif self.method == 'Word2Vec':
#             self.max_features = max_features
#             self.emb_fname = emb_fname
#             self.word_index_fname = word_index_fname
#         else:
#             raise ValueError('Feature extraction method does not exist.')

#     def feature_extraction(self, X_train, X_test):
#         train_data = self.vectorizer.fit_transform(X_train).toarray()
#         test_data = self.vectorizer.transform(X_test).toarray()
#         return train_data, test_data

#     def get_word_index(self):
#         word2id = json.load(open(self.word_index_fname, 'r'))
#         return word2id

#     def get_embedding_matrix(self):
#         np.random.seed(0)
#         word2id = self.get_word_index()
#         embedding_matrix = np.random.uniform(-0.25, 0.25, [len(word2id) + 1, self.max_features])
#         with open(self.emb_fname, 'r', encoding='utf-8') as f:
#             for line in f:
#                 content = line.split(' ')
#                 if content[0] in word2id:
#                     embedding_matrix[word2id[content[0]]] = np.array(list(map(float, content[1:])))
#         return embedding_matrix

In [11]:
# vectorizer = Vectorizer('BOW')
# X_train, X_test = vectorizer.feature_extraction(X_source, X_target)

In [12]:

# sentences = [sentence.split() for sentence in X_source]
# w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [13]:
# def vectorize(sentence):
#     words = sentence.split()
#     words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
#     if len(words_vecs) == 0:
#         return np.zeros(100)
#     words_vecs = np.array(words_vecs)
#     return words_vecs.mean(axis=0)

# X_train = np.array([vectorize(sentence) for sentence in X_source])
# X_test = np.array([vectorize(sentence) for sentence in X_target])

In [14]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [35]:
input_layer = X_train.shape[1]
input_size = input_layer
hidden_size =64

In [81]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_layer, 512),
            # nn.ReLU(),
            # nn.Dropout(0.3),
            # nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), input_layer)
        output = self.model(x)
        return output

In [83]:
discriminator = Discriminator().to(device=device)

In [84]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_layer, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, input_layer),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), input_layer)
        return output

generator = Generator().to(device=device)

In [85]:
lr = 0.00001
num_epochs = 1000
batch_size= 32
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [86]:
X_source = X_train.to(device=device, dtype=torch.float32)
X_target = X_test.to(device=device, dtype=torch.float32)


In [21]:
# X_source = torch.from_numpy(X_train).to(device=device, dtype=torch.float32)
# X_target = torch.from_numpy(X_test).to(device=device, dtype=torch.float32)

In [87]:
for epoch in range(num_epochs):
    idx = np.random.randint(0, X_source.shape[0], batch_size)
    real_samples  = X_source[idx]
    real_samples_labels = torch.ones((batch_size,1)).to(device=device)
    latent_space_samples = torch.rand((batch_size,input_layer)).to( device=device)
    
    generated_samples = generator(latent_space_samples)
    generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )
    all_samples = torch.cat((real_samples, generated_samples))
    all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
    discriminator.zero_grad()
    
    output_discriminator = discriminator(all_samples)
    all_samples_labels = all_samples_labels.view(-1, 1)
    loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
    loss_discriminator.backward()
    optimizer_discriminator.step()

        # Data for training the generator
    latent_space_samples = torch.rand((batch_size,input_layer)).to(device=device)

        # Training the generator
    generator.zero_grad()
    generated_samples = generator(latent_space_samples)
    output_discriminator_generated = discriminator(generated_samples)
    loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
    loss_generator.backward()
    optimizer_generator.step()
    if(epoch%100==0):
        print(f"Epoch: {epoch/100} Loss D.: {loss_discriminator}")
        print(f"Epoch: {epoch/100} Loss G.: {loss_generator}")
 

Epoch: 0.0 Loss D.: 0.6833779811859131
Epoch: 0.0 Loss G.: 0.6744371056556702
Epoch: 1.0 Loss D.: 0.5643945932388306
Epoch: 1.0 Loss G.: 0.5600152015686035
Epoch: 2.0 Loss D.: 0.5812645554542542
Epoch: 2.0 Loss G.: 0.5743303298950195
Epoch: 3.0 Loss D.: 0.5351682305335999
Epoch: 3.0 Loss G.: 0.6797955632209778
Epoch: 4.0 Loss D.: 0.48746925592422485
Epoch: 4.0 Loss G.: 0.6629167199134827
Epoch: 5.0 Loss D.: 0.4519200026988983
Epoch: 5.0 Loss G.: 0.6935461163520813
Epoch: 6.0 Loss D.: 0.46945640444755554
Epoch: 6.0 Loss G.: 0.6937960982322693
Epoch: 7.0 Loss D.: 0.42401832342147827
Epoch: 7.0 Loss G.: 0.7102949619293213
Epoch: 8.0 Loss D.: 0.4297463595867157
Epoch: 8.0 Loss G.: 0.6816540956497192
Epoch: 9.0 Loss D.: 0.4027346074581146
Epoch: 9.0 Loss G.: 0.6937461495399475


In [88]:
output_discriminator.shape

torch.Size([64, 1])

In [89]:
predicted_labels = discriminator(X_target)
new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))

In [90]:
new_label = new_tensor.numpy()

In [91]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy = accuracy_score(y_target, new_label)
accuracy

0.6468937364459753

In [93]:
print('accuracy =', accuracy_score(y_target, new_label))
print(classification_report(y_target,new_label))
print('Confusion Matrix: \n',confusion_matrix(y_target, new_label))

accuracy = 0.6468937364459753
              precision    recall  f1-score   support

           0       0.02      0.00      0.00     10948
           1       0.65      0.99      0.79     20408

    accuracy                           0.65     31356
   macro avg       0.33      0.50      0.39     31356
weighted avg       0.43      0.65      0.51     31356

Confusion Matrix: 
 [[    2 10946]
 [  126 20282]]
